In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten
import pydot

In [2]:
#Importing the Data
digit_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [16]:
digit_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
X_train = digit_data.drop('label', axis=1)
y_train = digit_data['label']

In [23]:
y_train.head()

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64

In [3]:
class_names = [0,1,2,3,4,5,6,7,8,9]

In [8]:
model = keras.models.Sequential([
    Flatten(input_shape=[784]),
    Dense(300,activation='relu'),
    Dense(100,activation='relu'),
    Dense(10,activation='softmax')
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.layers

In [11]:
model.layers[1].name

'dense'

In [12]:
weights, biases = model.layers[1].get_weights()

In [13]:
weights.shape

(784, 300)

In [14]:
biases.shape

(300,)

In [15]:
model.compile(loss='sparse_categorical_crossentropy',
    optimizer='sgd',
    metrics=['accuracy'])

In [25]:
digits = model.fit(X_train, y_train,
    validation_split=0.3)

1050/1050 [==============================] - 2s 2ms/step - loss: 2.3014 - accuracy: 0.1112 - val_loss: 6.8430 - val_accuracy: 0.1130


In [ ]:
#Josh making a change
print('Josh making a change.')